Проведите RFM анализ. В каждом подсегменте поделите пользователей на 4 класса. Отсчитывайте количество дней, прошедших с момента последней покупки, с максимальной даты покупки в датасете.  

- Какое максимальное кол-во покупок было совершено одним пользователем?
- Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)
- Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?
- Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R?
- Сколько пользователей попало в сегмент 111?
- Сколько пользователей попало в сегмент 311?
- В каком RFM-сегменте самое большое кол-во пользователей?
- В каком RFM-сегменте самое маленькое кол-во пользователей?
- Какое количество пользователей попало в самый малочисленный сегмент?

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/vitsli/DataAnalitycs/product_analitycs/RFM_ht_data.csv',
                 dtype={'CustomerCode': str}) 

In [4]:
df.head(10)

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00
5,C0011810010026,13163946,2020-09-01,160.00
6,C0011810010033,13233767,2020-09-01,372.37
7,C0011810010040,13233754,2020-09-01,222.62
8,C0011810010053,80007255,2020-09-01,1522.00
9,C0011810010065,13233771,2020-09-01,487.46


In [8]:
(df.groupby('CustomerCode', as_index=False)
     .agg({'InvoiceNo': 'count'})
     .rename(columns={'InvoiceNo':'OrderQuantity'})
     .sort_values('OrderQuantity')
)

,CustomerCode,OrderQuantity
0,02213019,1
67249,18053075,1
67247,18053072,1
67245,18053055,1
67243,18053019,1
...,...,...
119951,99003061,90
97077,19080880,99
10347,13032521,106
44594,13215452,113


In [9]:
df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [10]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [11]:
df['InvoiceNo'].apply('str')

In [12]:
maxdate = df['InvoiceDate'].max()
maxdate

Timestamp('2020-09-30 00:00:00')

In [13]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [14]:
rfm_table = df.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (maxdate - x.max()).days,
                                'InvoiceNo'  : lambda x: len(x),
                                'Amount'     : lambda x: x.sum()                            
                               }).copy()

In [15]:
rfm_table = rfm_table.rename(columns={'InvoiceDate': 'recency', 'InvoiceNo': 'frequency', 'Amount': 'monetary'})

In [16]:
rfm_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123733 entries, 02213019 to 99099972
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   recency    123733 non-null  int64  
 1   frequency  123733 non-null  int64  
 2   monetary   123733 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 3.8+ MB


In [27]:
rfm_table.shape[0] == df.CustomerCode.nunique()

True

In [28]:
quantiles = rfm_table.quantile(q=(0.25, 0.5, 0.75))

In [29]:
quantiles

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [30]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4

def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [31]:
rfm_table['R_Quartile'] = rfm_table['recency'].apply(RClass, args=('recency',quantiles))

rfm_table['F_Quartile'] = rfm_table['frequency'].apply(FMClass, args=('frequency',quantiles))

rfm_table['M_Quartile'] = rfm_table['monetary'].apply(FMClass, args=('monetary',quantiles))

rfm_table['RFMClass'] = (
                        rfm_table.R_Quartile.map(str) 
                        + rfm_table.F_Quartile.map(str)
                        + rfm_table.M_Quartile.map(str)
                        )

In [32]:
rfm_table.head(10)

,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443
02213128,24,2,2681.75,4,3,2,432
02213129,1,4,2199.00,1,1,2,112
02213155,27,1,1667.40,4,4,3,443
02213163,5,1,328.76,2,4,4,244


In [34]:
print('Максимальное количество покупок, совершенное одним пользователем равно', rfm_table['frequency'].max())

Максимальное количество покупок, совершенное одним пользователем равно 204


In [37]:
print('Верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М равна',
      rfm_table[rfm_table.M_Quartile == 4].monetary.max())

Верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М равна 765.0


In [39]:
print('Нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F равна', 
      rfm_table[rfm_table.F_Quartile == 1].frequency.min())

Нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F равна 4


In [48]:
print(f'Верхняя граница с момента последней покупки \
для 2-го класса в подсегменте R составляет \
{rfm_table[rfm_table.R_Quartile == 2].recency.max()} дней')

Верхняя граница с момента последней покупки для 2-го класса в подсегменте R составляет 8 дней


In [53]:
print('Количество пользователей в сегменте 111 равно', rfm_table[rfm_table.RFMClass == '111'].shape[0])

Количество пользователей в сегменте 111 равно 9705


In [55]:
print('Количество пользователей в сегменте 311 равно', rfm_table[rfm_table.RFMClass == '311'].shape[0])

Количество пользователей в сегменте 311 равно 1609


In [58]:
rfm_table.RFMClass.value_counts()

444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
114       60
214       60
314       33
414        2
Name: RFMClass, Length: 64, dtype: int64

Наибольшее количество пользователей в сегменте 444, наименьшее - в 414